In [1]:
import numpy as np
from warcio.archiveiterator import ArchiveIterator
import re
import requests

In [ ]:
main_url = "http://commoncrawl.s3.amazonaws.com/"
counter = 0

with open("warc.paths", "r") as infile:
    files = infile.readlines()

#randomly select some file to go through
visited = {}
while counter <= 1000:
    file = np.random.choice(files)
    while file in visited:
        file = np.random.choice(files)
    visited[file] = True

    file_name = main_url + file.strip()

    stream = requests.get(file_name, stream=True).raw


    #english_regex = "<meta.*en-us.*>"
    #news_regex = "(bbc)|(cnn)|(abc)|(wsj)|(boloomberg)|(vice)|(washingtonpost)|(theguardian)"
    covid_regex = "COVID[-]*19"
    covid_meta_regex = "<meta.*keywords.*covid"
    enconomic_regex = "econom"

    for record in ArchiveIterator(stream):
        if record.rec_type == "warcinfo":
            continue

        if not ".com/" in record.rec_headers.get_header(
            "WARC-Target-URI"
        ):
            continue

        curr_url = record.rec_headers.get_header("WARC-Target-URI")

        #if len(re.findall(news_regex, curr_url, re.IGNORECASE)) < 1:
        #    continue

        contents = (
            record.content_stream()
            .read()
            .decode("utf-8", "replace")
        )

        #if len(re.findall(english_regex, contents, re.IGNORECASE)) < 1:
        #    continue

        if len(re.findall(covid_meta_regex, contents, re.IGNORECASE)) < 1:
            continue
        
        if len(re.findall(covid_regex, contents, re.IGNORECASE)) < 50:
            continue

        if len(re.findall(enconomic_regex, contents, re.IGNORECASE)) < 50:
            continue

        print(counter)
        print(curr_url)
        #print(re.findall(covid_meta_regex, contents, re.IGNORECASE))
        #print(re.findall(covid_regex, contents), len(re.findall(covid_regex, contents, re.IGNORECASE)))
        #print(re.findall(enconomic_regex, contents, re.IGNORECASE), len(re.findall(enconomic_regex, contents, re.IGNORECASE)))

        with open("pages2.txt", "a+") as outfile:
            outfile.write(str(curr_url))
            outfile.write(",")

        counter += 1